<a href="https://colab.research.google.com/github/Indranil0603/meesho-dice-challenge-2025/blob/master/etl_pipeline_refund_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ETL pip

In [ ]:
# Making necessary imports
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
# Reading the data
df = pd.read_excel("/content/drive/MyDrive/amazon_datasets/amazon_orders.xlsx")

In [ ]:
# Data Visualization
df.head()

,Order_ID,Order_Date,Shipment_ID,Shipment_Date,Posted_Date,Fulfillment,ASIN,SKU,Transaction_Type,Tax_Collection_Model,...,Ship_To_Postal_Code,Ship_To_Location_Code,Taxed_Location_Code,Tax_Address_Role,Jurisdiction_Level,Jurisdiction_Name,Tax_Amount_Collected_By_Amazon,Taxed_Jurisdiction_Tax_Rate,Tax_Type,Taxable_Amount
0,111-0000314-6295456,2021-01-04+00:00,Dszmsmvlz,2021-01-05+00:00,2021-01-05+00:00,MFN,B073XXCKNK,3L-M88Q-2QNW,SHIPMENT,MarketplaceFacilitator,...,48316,230990000,230990000,ShipTo,State,MI,3.12,0.06,Sales and Use Tax,51.94
1,111-0000967-4355409,2021-01-26+00:00,DWjjW1qgp,2021-01-27+00:00,2021-01-27+00:00,AFN,B073XXX1FF,3Y0-IIA-XEY,SHIPMENT,MarketplaceFacilitator,...,35801-1300,10890580,10890580,ShipTo,State,AL,4.72,0.08,Sales and Use Tax,58.94
2,111-0000967-4355409,2021-01-26+00:00,DWjjW1qgp,2021-01-27+00:00,2021-01-27+00:00,AFN,B07PN8Q7HX,LR-G6QR-2UC4,SHIPMENT,MarketplaceFacilitator,...,35801-1300,10890580,10890580,ShipTo,State,AL,5.11,0.08,Sales and Use Tax,63.93
3,111-0000967-4355409,2021-01-26+00:00,DWjjW1qgp,2021-02-10+00:00,2021-02-10+00:00,AFN,B073XXX1FF,3Y0-IIA-XEY,RETURN,MarketplaceFacilitator,...,35801-1300,10890580,10890580,ShipTo,State,AL,-4.72,0.08,Sales and Use Tax,-58.94
4,111-0000967-4355409,2021-01-26+00:00,DWjjW1qgp,2021-02-10+00:00,2021-02-10+00:00,AFN,B07PN8Q7HX,LR-G6QR-2UC4,RETURN,MarketplaceFacilitator,...,35801-1300,10890580,10890580,ShipTo,State,AL,-5.11,0.08,Sales and Use Tax,-63.93


In [ ]:
# Data Date Range

In [ ]:
df['Order_Date'].min()

'2020-03-08+00:00'

In [ ]:
df['Order_Date'].max()

'2021-12-31+00:00'

In [ ]:
# Applying date operations
df['Order_Date'] = df['Order_Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d%z'))
df['Shipment_Date'] = df['Shipment_Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d%z'))
df['Posted_Date'] = df['Posted_Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d%z'))

In [ ]:
# Generating Customer and Sellers Approximations
num_months = 22
avg_orders_per_day = 8.84*1e6 # Source: Google
total_orders = 8.84*1e6*(306+365)
orders_count = len(df)
sampling_factor = orders_count/total_orders

avg_orders_per_customer_per_month = 3 # Source: Google
orders_per_customer = avg_orders_per_customer_per_month * num_months
customers_count = int((total_orders/orders_per_customer)*sampling_factor)
customers_count += int(0.25*customers_count)

amazon_sellers_count = 2*1e6 # Source: Google
sellers_count = int(amazon_sellers_count*sampling_factor)
sellers_count += 50 # Base sellers

In [ ]:
# Approx customer count
customers_count

347

In [ ]:
# Approx seller count
sellers_count

56

In [ ]:
# Assigning synthetic customers and sellers only for reference
np.random.seed(42)
df["Customer_ID"] = np.random.choice([f"CUST_{i:05d}" for i in range(1, customers_count+1)], size=len(df))
np.random.seed(42)
df["Seller_ID"]   = np.random.choice([f"SELL_{i:04d}" for i in range(1, sellers_count+1)], size=len(df))

In [ ]:
# Checking columns
df.columns

Index(['Order_ID', 'Order_Date', 'Shipment_ID', 'Shipment_Date', 'Posted_Date',
       'Fulfillment', 'ASIN', 'SKU', 'Transaction_Type',
       'Tax_Collection_Model', 'Quantity', 'Display_Price',
       'TaxExclusive_Selling_Price', 'Total_Tax_Collected_By_Amazon',
       'Financial_Component', 'Ship_From_City', 'Ship_From_State',
       'Ship_From_Country', 'Ship_From_Postal_Code',
       'Ship_From_Tax_Location_Code', 'Ship_To_City', 'Ship_To_State',
       'Ship_To_Country', 'Ship_To_Postal_Code', 'Ship_To_Location_Code',
       'Taxed_Location_Code', 'Tax_Address_Role', 'Jurisdiction_Level',
       'Jurisdiction_Name', 'Tax_Amount_Collected_By_Amazon',
       'Taxed_Jurisdiction_Tax_Rate', 'Tax_Type', 'Taxable_Amount',
       'Customer_ID', 'Seller_ID'],
      dtype='object')

In [ ]:
# Filtering Columns
cols = ['Order_ID', 'Order_Date', 'Shipment_ID', 'Shipment_Date', 'Posted_Date', 'SKU', 'Transaction_Type',
        'Quantity', 'Ship_From_Postal_Code', 'Ship_To_Postal_Code', 'Customer_ID', 'Seller_ID']
df = df[cols]

In [ ]:
# Analyzig returns and refunds
df['Transaction_Type'].value_counts()

,count
Transaction_Type,
SHIPMENT,16578
RETURN,1504
REFUND,187
CHARGE,115
RETROCHARGE,19


In [ ]:
# Filtering products with refunds and returns
df_product_problem = df[(df['Transaction_Type'] == 'RETURN') | (df['Transaction_Type'] == 'Refund')].reset_index(drop=True)

In [ ]:
# Filtering problematic products
df_sku_problems = df_product_problem.groupby('SKU').agg(
    order_ids = ('Order_ID', lambda x: list(x)),
    count = ('Order_ID', 'count'),
    seller_ids = ('Seller_ID', lambda x: list(x))
)

In [ ]:
# Visualizing Data
df_sku_problems.head()

,order_ids,count,seller_ids
SKU,,,
1Z-B5YE-Q4Z5,"[111-2433181-9245067, 111-3007243-7625813, 111...",63,"[SELL_0017, SELL_0025, SELL_0037, SELL_0019, S..."
"3"" Hollow balls Qty2","[111-2137508-0421049, 112-0494897-1973045, 112...",3,"[SELL_0010, SELL_0022, SELL_0032]"
3Y0-IIA-XEY,"[111-0000967-4355409, 111-0086816-1599444, 111...",401,"[SELL_0015, SELL_0015, SELL_0055, SELL_0044, S..."
5E-U7K9-X81Z,"[111-0220768-3409854, 111-1457110-9605829, 111...",14,"[SELL_0012, SELL_0035, SELL_0023, SELL_0014, S..."
92-NWVN-FXUA,"[111-0268222-6625069, 111-3073110-2462669, 111...",10,"[SELL_0015, SELL_0001, SELL_0020, SELL_0010, S..."


In [ ]:
# Net cases
len(df_sku_problems)

17

In [ ]:
# Analyziing the sku count distribution
df_sku_problems['count'].describe()

,count
count,17.000000
mean,88.470588
std,107.591890
min,1.000000
25%,10.000000
50%,54.000000
75%,122.000000
max,401.000000


In [ ]:
# Sku Counts
df_sku_counts = df.groupby('SKU').agg(
    order_ids = ('Order_ID', lambda x: list(x)),
    net_count = ('Order_ID', 'count')
)

df_sku_problems = pd.merge(df_sku_problems, df_sku_counts, on='SKU', how='left')

In [ ]:
# Computing return/refund ration
df_sku_problems['return_refund_ratio'] = df_sku_problems['count']/df_sku_problems['net_count']

In [ ]:
# Analyzing the distribution of return refund ratio
df_sku_problems['return_refund_ratio'].describe()

,return_refund_ratio
count,17.000000
mean,0.080997
std,0.037707
min,0.017730
25%,0.076923
50%,0.088577
75%,0.107325
max,0.138952


The return refund ratio between the 3rd quartile and 4th quartile liew between 10%-14%.
Hence a threshold after further analysis for return refund ratio can be used for product re-verification and
sending the seller a notification for inspecting the product

In [ ]:
# Visualizing Sellers with Faulty Products
df_seller_problems = df_product_problem.groupby('Seller_ID').agg(
    order_ids = ('Order_ID', lambda x: list(x)),
    count = ('Order_ID', 'count'),
    sku_ids = ('SKU', lambda x: list(x))
)

In [ ]:
# Visualizing Data
df_seller_problems.head()

,order_ids,count,sku_ids
Seller_ID,,,
SELL_0001,"[111-1103497-2762656, 111-3073110-2462669, 111...",27,"[3Y0-IIA-XEY, 92-NWVN-FXUA, Blk 10"" 2 Pack, Bl..."
SELL_0002,"[111-3264587-3417860, 111-3629364-2686668, 111...",26,"[3Y0-IIA-XEY, Blk 12"" 2 Pack, Blk 12"" 2 Pack, ..."
SELL_0003,"[111-1254512-8979427, 111-1961134-3733821, 111...",25,"[WHT 3 FBA, 3Y0-IIA-XEY, Blk 12"" 2 Pack, LR-G6..."
SELL_0004,"[111-3289671-9784228, 111-4824758-6883450, 111...",31,"[3Y0-IIA-XEY, LR-G6QR-2UC4, WHT 4 FBA, Blk 12""..."
SELL_0005,"[111-1537118-5366661, 111-2060073-3433815, 111...",35,"[WHT 4 FBA, 3Y0-IIA-XEY, FBA3L-M88Q-2QNW, Blk ..."


In [ ]:
# Net cases
len(df_seller_problems)

56

In [ ]:
# Net Orders per Seller
df_sellers = df.groupby('Seller_ID').agg(
    order_ids = ('Order_ID', lambda x: list(x)),
    net_count = ('Order_ID', 'count')
)

In [ ]:
df_seller_problems = pd.merge(df_seller_problems, df_sellers, on='Seller_ID', how='left')
df_seller_problems['return_refund_ratio'] = df_seller_problems['count']/df_seller_problems['net_count']

In [ ]:
df_seller_problems['return_refund_ratio'].describe()

,return_refund_ration
count,56.000000
mean,0.081763
std,0.015997
min,0.041667
25%,0.071150
50%,0.080286
75%,0.090404
max,0.121495


In [ ]:
# Filtering Customers with frequent returns and refunds
df_customer_problems = df_product_problem.groupby('Customer_ID').agg(
    order_ids = ('Order_ID', lambda x: list(x)),
    count = ('Order_ID', 'count')
)

In [ ]:
# Visualizing Data
df_customer_problems.head()

,order_ids,count
Customer_ID,,
CUST_00001,"[111-4175080-0991438, 112-2414915-4908265]",2
CUST_00002,"[111-4044165-3424215, 112-8038106-1277013, 114...",5
CUST_00003,"[111-7686962-9420268, 111-9223329-2286646, 112...",6
CUST_00004,"[112-1819152-2240243, 112-5563458-0541815, 112...",9
CUST_00005,"[111-3277615-7945818, 111-8710666-4967463, 113...",6


In [ ]:
# Net cases
len(df_customer_problems)

345

In [ ]:
df_customers = df.groupby('Customer_ID').agg(
    order_ids = ('Order_ID', lambda x: list(x)),
    net_count = ('Order_ID', 'count')
)

df_customer_problems = pd.merge(df_customer_problems, df_customers, on='Customer_ID', how='left')
df_customer_problems['return_refund_ratio'] = df_customer_problems['count']/df_customer_problems['net_count']
df_customer_problems['return_refund_ratio'].describe()

,return_refund_ration
count,345.000000
mean,0.082112
std,0.038591
min,0.014925
25%,0.054545
50%,0.078125
75%,0.105263
max,0.210526


In [ ]:
# Mimicing Delivery Delays
delayed_orders_ratio = 0.15 # Source: Google
df['delivery_delayed'] = 'NO'

sample_size = int(len(df) * delayed_orders_ratio)
sample_indices = df.sample(n=sample_size, random_state=42).index
df.loc[sample_indices, 'delivery_delayed'] = 'YES'

np.random.seed(42)
df['delivery_delay'] = np.random.choice([-3,-2,-1,0], size=len(df))
df.loc[sample_indices, 'delivery_delay'] = np.random.choice([1,2,3], size = sample_size)

In [ ]:
# Checking distribution
df['delivery_delay'].value_counts()

,count
delivery_delay,
-3,3993
-2,3908
0,3900
-1,3842
2,959
1,903
3,898


In [ ]:
# Filtering delayed orders
df_delivery_delays = df[df['delivery_delay'] > 0].reset_index(drop=True)

In [ ]:
# Filtering pincodes for delayed orders and average delay
df_warehouse_codes_delays = df_delivery_delays.groupby('Ship_To_Postal_Code').agg(
    order_ids = ('Order_ID', lambda x: list(x)),
    delay_count = ('Order_ID', 'count'),
    delivery_delay = ('delivery_delay', 'mean')
)

In [ ]:
df_warehouse_codes = df.groupby('Ship_To_Postal_Code').agg(
    order_ids = ('Order_ID', lambda x: list(x)),
    count = ('Order_ID', 'count')
)

df_warehouse_codes_delays = pd.merge(df_warehouse_codes_delays, df_warehouse_codes, on='Ship_To_Postal_Code', how='left')
df_warehouse_codes_delays['delay_delivery_ratio'] = df_warehouse_codes_delays['delay_count']/df_warehouse_codes_delays['count']


In [ ]:
# Visualizing Data
df_warehouse_codes_delays['delay_delivery_ratio'].describe()

,delay_delivery_ratio
count,2384.000000
mean,0.655140
std,0.313282
min,0.038462
25%,0.400000
50%,0.500000
75%,1.000000
max,1.000000


In [ ]:
# Net cases
len(df_warehouse_codes_delays)

2384

In [ ]:
# Net unique pincodes
df['Ship_To_Postal_Code'].nunique()

10490

An analysis similar to sku can be done for customer, seller, and delivery delay pincode counts and ratio